- Server address : `https://fb90-165-132-144-84.ngrok-free.app`
- Endpoints:
    - `/api/inference`: For calling API to generate outputs from LMs
    - `/api/change_password`: Change your password. Arguments: `student_id`, `old_password`, `new_password`.

- Check your remaining credit at this page.[https://fb90-165-132-144-84.ngrok-free.app/student_credits] 

### Before we get started, let's change the password!

In [147]:
import requests
import os

server_address = "https://cot.ngrok.app"
os.environ['INTRO_AI_API_KEY'] ='kimys0613'

In [148]:
class LLM:
    def __init__(self, model_name, student_id, password):
        self.student_id = student_id
        self.password = password
        server_address = "https://cot.ngrok.app"
        self.api_url = f'{server_address}/api/inference'  # Replace with your server address
        self.model_name = model_name

    def generate(self, model_input, max_tokens=100, top_p=1.0, temperature=0.7, frequency_penalty=0, stop="\n\n"):
        data = {
            'model_name': self.model_name,
            'model_input': model_input,
            'student_id': self.student_id,
            'password': self.password,
            'max_tokens': max_tokens,
            'top_p': top_p,
            'temperature': temperature,
            'frequency_penalty': frequency_penalty,
            'stop': stop
        }
        response = requests.post(self.api_url, json=data)

        if response.status_code == 200:
            return response.json()  # Assuming the server returns a JSON response
        else:
            return f"Error: {response.text}"

In [149]:
student_id = "2022148083"
password = os.environ['INTRO_AI_API_KEY']

available_models = [
    "meta-llama/Llama-2-7b-hf",
    "facebook/opt-350m",
    "facebook/opt-1.3b",
    "facebook/opt-2.7b"
]
opt = LLM(available_models[1], student_id, password)

### Generate your first output by calling the API

In [150]:
model_input = "Hello World!"
result = opt.generate([model_input], stop=None)

In [151]:
print(result)

{'generations': [{'generation_info': {'finish_reason': 'stop', 'logprobs': None}, 'text': '\n\nWelcome to our website!\n\nWelcome to our website! We are constantly working to make our site better by helping you find what you’re looking for. If you have any questions you should have a chat with us. We’ll be happy to help.'}], 'llm_output': {'model_name': 'facebook/opt-350m', 'token_usage': {'completion_tokens': 58, 'prompt_tokens': 4, 'total_tokens': 62}}, 'run_info': [{'run_id': '6690b6aa-e924-4fb9-83f6-73b817d32f6c'}]}


### Load GSM8K dataset

In [152]:
from datasets import load_dataset

gsm8k = load_dataset("gsm8k", "main")['test']

### An example from GSM8k

In [153]:
print("Question:")
print(gsm8k['question'][0])
print("="*100)
print("Answer:")
print(gsm8k['answer'][0])

Question:
Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
Answer:
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


## **Experiment 1**: Zero-shot prompting v.s. Few-shot prompting

### Constructing prompts

In [154]:
import random
random.seed(0)

def construct_prompt(num_exemplars):
    # Load train set of GSM8K1
    gsm8k = load_dataset("gsm8k", "main")['train']

    sampled_indices = random.sample([i for i in range(len(gsm8k['question']))], num_exemplars)


    instruction = "Instruction:\nSolve the following mathematical question and generate the answer after a tag, 'Answer:'."

    # Constructing a prompt with few-shot demonstrations from GSM8K
    prompt = instruction
    for i in range(num_exemplars):
        cur_question = gsm8k['question'][i]
        cur_answer = gsm8k['answer'][i].split("####")[-1].strip()
        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += f"\n[Example {num_exemplars+1}]\n"
    prompt += "Question:\n{question}\nAnswer:"

    # Write the prompt to a .txt file
    with open(f"prompt_{num_exemplars}shot.txt", "w") as f:
        f.write(prompt)



In [155]:
construct_prompt(0)
construct_prompt(3)

### Load the prompt for zero-shot prompting

In [156]:
with open("prompt_0shot.txt", "r") as f:
    prompt_0shot = f.read()

In [161]:
from tqdm import tqdm
import json

llama = LLM("meta-llama/Llama-2-7b-hf", student_id, password)
results_collected = []
pass_collected = []
for i in tqdm(range(50)):
    cur_question = gsm8k['question'][i]
    cur_anwer = gsm8k['answer'][i].split("####")[-1].strip()
    cur_model_input = prompt_0shot.format(question=cur_question)
    result = llama.generate([cur_model_input], stop=["\n\n", "[Example 2]"])
    if "Error:" in result:
        print(result)
        break
    cur_prediction = result['generations'][0]['text']
    # print(cur_prediction)

    pass_collected.append(cur_prediction.strip().replace("$", "0") == cur_anwer)
    results_collected.append({"question": cur_question, "answer": cur_anwer, "prediction": cur_prediction})
    print(f"Acc: {sum(pass_collected)/ len(pass_collected)}")

with open("result_0shot_direct.json", "w") as f:
    json.dump(results_collected, f, indent=4)

  2%|▏         | 1/50 [00:00<00:42,  1.15it/s]

Acc: 0.0


  4%|▍         | 2/50 [00:01<00:30,  1.59it/s]

Acc: 0.5


  6%|▌         | 3/50 [00:01<00:24,  1.92it/s]

...

Acc: 0.020833333333333332


 98%|█████████▊| 49/50 [00:27<00:00,  1.35it/s]

Acc: 0.02040816326530612


100%|██████████| 50/50 [00:28<00:00,  1.76it/s]

Acc: 0.02


In [163]:
with open("prompt_3shot.txt", "r") as f:
    prompt_3shot = f.read()
    
from tqdm import tqdm

llama = LLM("meta-llama/Llama-2-7b-hf", student_id, password)
results_collected = []
pass_collected = []
for i in tqdm(range(50)):
    cur_question = gsm8k['question'][i]
    cur_anwer = gsm8k['answer'][i].split("####")[-1].strip()
    cur_model_input = prompt_3shot.format(question=cur_question)
    result = llama.generate([cur_model_input], stop=["\n\n"])
    if "Error:" in result:
        print(result)
        break
    cur_prediction = result['generations'][0]['text']
    # print(cur_prediction)

    pass_collected.append(cur_prediction.strip().replace("$","") == cur_anwer)
    results_collected.append({"question": cur_question, "answer": cur_anwer, "prediction": cur_prediction})
    print(f"Acc: {sum(pass_collected)/ len(pass_collected)}")
    
with open("result_3shot_direct.json", "w") as f:
    json.dump(results_collected, f, indent=4)

  2%|▏         | 1/50 [00:00<00:20,  2.36it/s]

Acc: 0.0


  4%|▍         | 2/50 [00:00<00:18,  2.62it/s]

Acc: 0.5


  6%|▌         | 3/50 [00:01<00:20,  2.34it/s]

...

Acc: 0.0625


 98%|█████████▊| 49/50 [00:20<00:00,  2.38it/s]

Acc: 0.061224489795918366


100%|██████████| 50/50 [00:20<00:00,  2.40it/s]

Acc: 0.06


## **Experiment 2**: CoT prompting vs Standard prompting

In [164]:
import random

def generate_chain_of_thought(prompt, num_chains=2):
    chains = []

    for _ in range(num_chains):
        # Create a chain of thought
        chain = ""
        tokens = prompt.split()
        for i in range(len(tokens)):
            # Randomly decide whether to include each token in the chain
            if random.choice([True, False]):
                chain += tokens[i] + " "
        chains.append(chain.strip())

    return chains

def construct_chain_of_thought_prompt(num_chains):
    # Load train set of GSM8K1
    gsm8k = load_dataset("gsm8k", "main")['train']

    sampled_indices = random.sample([i for i in range(len(gsm8k['question']))], num_chains)

    instruction = "Instruction:\nSolve the following mathematical question and generate the answer after a tag, 'Answer:'."
    prompt = instruction

    for i in range(num_chains):
        cur_question = gsm8k['question'][i]
        cur_answer = gsm8k['answer'][i].split("####")[-1].strip()

        # Generate chain of thought for each example
        chains = generate_chain_of_thought(f"{cur_question} Answer: {cur_answer}", num_chains=2)

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        
        for j, chain in enumerate(chains):
            prompt += f"Chain of Thought {j+1}:\n{chain}\n"

        prompt += f"Answer:{cur_answer}\n"

    # Write the prompt to a .txt file
    with open(f"prompt_chain_of_thought_{num_chains}chains.txt", "w") as f:
        f.write(prompt)


In [165]:
construct_chain_of_thought_prompt(3)

In [ ]:
from tqdm import tqdm
import json

# Assuming gsm8k is a valid dataset containing questions and answers
gsm8k = load_dataset("gsm8k", "main")['train']

# Assuming prompt_chain_of_thought_3chains.txt is the file you created using the provided code
with open("prompt_chain_of_thought_3chains.txt", "r") as f:
    prompt_chain_of_thought = f.read()

llama = LLM("meta-llama/Llama-2-7b-hf", student_id, password)
results_collected = []
pass_collected = []

# Number of chains of thought to consider for each example
num_chains = 2

for i in tqdm(range(50)):
    cur_question = gsm8k['question'][i]
    cur_answer = gsm8k['answer'][i].split("####")[-1].strip()
    cur_model_input = prompt_chain_of_thought.format(question=cur_question)
    
    # Generate chains of thought
    chains_of_thought = generate_chain_of_thought(cur_question, num_chains)
    
    # Combine chains of thought with the original model input
    cur_model_input_with_chains = f"{cur_model_input}\n\n"
    for j, chain in enumerate(chains_of_thought):
        cur_model_input_with_chains += f"Chain of Thought {j + 1}:\n{chain}\n"
    
    # Assuming generate method of LLM class supports a list of inputs
    result = llama.generate([cur_model_input_with_chains], stop=["\n\n", "[Example 4]"])
    
    if "Error:" in result:
        print(result)
        break
    
    # Extracting the generated answer from the result
    generated_text = result['generations'][0]['text']
    
    # Check if "Answer:" is present in the generated text
    if "Answer:" in generated_text:
        cur_prediction = generated_text.split("Answer:")[1].strip()
        
        # Compare the prediction with the answer
        pass_collected.append(cur_prediction.strip().replace("$", "") == cur_answer)
        
        results_collected.append({
            "question": cur_question,
            "answer": cur_answer,
            "prediction": f"Answer:{cur_prediction}"
        })
    else:
        print(f"Error: 'Answer:' not found in the generated text.")
    
    print(f"Acc: {sum(pass_collected) / len(pass_collected)}")

with open("result_chain_of_thought_3shot_direct.json", "w") as f:
    json.dump(results_collected, f, indent=4)


## **Experiment 3**: Applying CoT prompting on models of various sizes

In [168]:
from tqdm import tqdm
import json

# Assuming gsm8k is a valid dataset containing questions and answers
gsm8k = load_dataset("gsm8k", "main")['train']

# Assuming prompt_chain_of_thought_3chains.txt is the file you created using the provided code
with open("prompt_chain_of_thought_3chains.txt", "r") as f:
    prompt_chain_of_thought = f.read()

llama = LLM("meta-llama/Llama-2-7b-hf", student_id, password)
results_collected = []
pass_collected = []

# Number of chains of thought to consider for each example
num_chains = 2

for i in tqdm(range(50)):
    cur_question = gsm8k['question'][i]
    cur_answer = gsm8k['answer'][i].split("####")[-1].strip()
    cur_model_input = prompt_chain_of_thought.format(question=cur_question)
    
    # Generate chains of thought
    chains_of_thought = generate_chain_of_thought(cur_question, num_chains)
    
    # Combine chains of thought with the original model input
    cur_model_input_with_chains = f"{cur_model_input}\n\n"
    for j, chain in enumerate(chains_of_thought):
        cur_model_input_with_chains += f"Chain of Thought {j + 1}:\n{chain}\n"
    
    # Assuming generate method of LLM class supports a list of inputs
    result = llama.generate([cur_model_input_with_chains], stop=["\n\n", "[Example 4]"])
    
    if "Error:" in result:
        print(result)
        break
    
    # Extracting the generated answer from the result
    generated_text = result['generations'][0]['text']
    
    # Check if "Answer:" is present in the generated text
    if "Answer:" in generated_text:
        cur_prediction = generated_text.split("Answer:")[1].strip()
        
        # Compare the prediction with the answer
        pass_collected.append(cur_prediction.strip().replace("$", "") == cur_answer)
        
        results_collected.append({
            "question": cur_question,
            "answer": cur_answer,
            "prediction": f"Answer:{cur_prediction}"
        })
    else:
        print(f"Error: 'Answer:' not found in the generated text.")
    
    print(f"Acc: {sum(pass_collected) / len(pass_collected)}")

with open("LLaMA_CoT.json", "w") as f:
    json.dump(results_collected, f, indent=4)


  2%|▏         | 1/50 [00:01<01:34,  1.94s/it]

Acc: 1.0


  4%|▍         | 2/50 [00:02<00:52,  1.09s/it]

Acc: 0.5


  6%|▌         | 3/50 [00:02<00:37,  1.24it/s]

...

Acc: 0.023255813953488372


 98%|█████████▊| 49/50 [00:34<00:00,  1.48it/s]

Acc: 0.022727272727272728


100%|██████████| 50/50 [00:35<00:00,  1.41it/s]

Acc: 0.022222222222222223


In [179]:
from tqdm import tqdm
import json

# Assuming gsm8k is a valid dataset containing questions and answers
gsm8k = load_dataset("gsm8k", "main")['train']

# Assuming prompt_chain_of_thought_3chains.txt is the file you created using the provided code
with open("prompt_chain_of_thought_3chains.txt", "r") as f:
    prompt_chain_of_thought = f.read()

llama = LLM("facebook/opt-350m", student_id, password)
results_collected = []
pass_collected = []

# Number of chains of thought to consider for each example
num_chains = 2

for i in tqdm(range(50)):
    cur_question = gsm8k['question'][i]
    cur_answer = gsm8k['answer'][i].split("####")[-1].strip()
    cur_model_input = prompt_chain_of_thought.format(question=cur_question)
    
    # Generate chains of thought
    chains_of_thought = generate_chain_of_thought(cur_question, num_chains)
    
    # Combine chains of thought with the original model input
    cur_model_input_with_chains = f"{cur_model_input}\n\n"
    for j, chain in enumerate(chains_of_thought):
        cur_model_input_with_chains += f"Chain of Thought {j + 1}:\n{chain}\n"
    
    # Assuming generate method of LLM class supports a list of inputs
    result = llama.generate([cur_model_input_with_chains], stop=["\n\n", "[Example 4]"])
    
    if "Error:" in result:
        print(result)
        break
    
    # Extracting the generated answer from the result
    generated_text = result['generations'][0]['text']
    
    # Check if "Answer:" is present in the generated text
    if "Answer:" in generated_text:
        cur_prediction = generated_text.split("Answer:")[1].strip()
        
        # Compare the prediction with the answer
        pass_collected.append(cur_prediction.strip().replace("$", "") == cur_answer)
        
        results_collected.append({
            "question": cur_question,
            "answer": cur_answer,
            "prediction": f"Answer:{cur_prediction}"
        })
    else:
        print(f"Error: 'Answer:' not found in the generated text.")
    
    # Check if pass_collected is not empty before calculating accuracy
    if pass_collected:
        print(f"Acc: {sum(pass_collected) / len(pass_collected)}")

with open("opt-350m.json", "w") as f:
    json.dump(results_collected, f, indent=4)


  2%|▏         | 1/50 [00:01<01:00,  1.24s/it]

Error: 'Answer:' not found in the generated text.


  4%|▍         | 2/50 [00:02<00:49,  1.03s/it]

Acc: 0.0


  6%|▌         | 3/50 [00:02<00:33,  1.39it/s]

...

Error: 'Answer:' not found in the generated text.
Acc: 0.02564102564102564


 98%|█████████▊| 49/50 [00:31<00:00,  1.67it/s]

Acc: 0.025


100%|██████████| 50/50 [00:32<00:00,  1.55it/s]

Error: 'Answer:' not found in the generated text.
Acc: 0.025


In [176]:
from tqdm import tqdm
import json

# Assuming gsm8k is a valid dataset containing questions and answers
gsm8k = load_dataset("gsm8k", "main")['train']

# Assuming prompt_chain_of_thought_3chains.txt is the file you created using the provided code
with open("prompt_chain_of_thought_3chains.txt", "r") as f:
    prompt_chain_of_thought = f.read()

llama = LLM("facebook/opt-1.3b", student_id, password)
results_collected = []
pass_collected = []

# Number of chains of thought to consider for each example
num_chains = 2

for i in tqdm(range(50)):
    cur_question = gsm8k['question'][i]
    cur_answer = gsm8k['answer'][i].split("####")[-1].strip()
    cur_model_input = prompt_chain_of_thought.format(question=cur_question)
    
    # Generate chains of thought
    chains_of_thought = generate_chain_of_thought(cur_question, num_chains)
    
    # Combine chains of thought with the original model input
    cur_model_input_with_chains = f"{cur_model_input}\n\n"
    for j, chain in enumerate(chains_of_thought):
        cur_model_input_with_chains += f"Chain of Thought {j + 1}:\n{chain}\n"
    
    # Assuming generate method of LLM class supports a list of inputs
    result = llama.generate([cur_model_input_with_chains], stop=["\n\n", "[Example 4]"])
    
    if "Error:" in result:
        print(result)
        break
    
    # Extracting the generated answer from the result
    generated_text = result['generations'][0]['text']
    
    # Check if "Answer:" is present in the generated text
    if "Answer:" in generated_text:
        cur_prediction = generated_text.split("Answer:")[1].strip()
        
        # Compare the prediction with the answer
        pass_collected.append(cur_prediction.strip().replace("$", "") == cur_answer)
        
        results_collected.append({
            "question": cur_question,
            "answer": cur_answer,
            "prediction": f"Answer:{cur_prediction}"
        })
    else:
        print(f"Error: 'Answer:' not found in the generated text.")
    
    print(f"Acc: {sum(pass_collected) / len(pass_collected)}")

with open("opt-1.3b.json", "w") as f:
    json.dump(results_collected, f, indent=4)


  2%|▏         | 1/50 [00:00<00:18,  2.66it/s]

Acc: 1.0


  4%|▍         | 2/50 [00:00<00:15,  3.20it/s]

Acc: 0.5


  6%|▌         | 3/50 [00:01<00:34,  1.36it/s]

...

Error: 'Answer:' not found in the generated text.
Acc: 0.029411764705882353


 98%|█████████▊| 49/50 [00:26<00:00,  1.72it/s]

Acc: 0.02857142857142857


100%|██████████| 50/50 [00:27<00:00,  1.83it/s]

Acc: 0.027777777777777776


In [178]:
from tqdm import tqdm
import json

# Assuming gsm8k is a valid dataset containing questions and answers
gsm8k = load_dataset("gsm8k", "main")['train']

# Assuming prompt_chain_of_thought_3chains.txt is the file you created using the provided code
with open("prompt_chain_of_thought_3chains.txt", "r") as f:
    prompt_chain_of_thought = f.read()

llama = LLM("facebook/opt-2.7b", student_id, password)
results_collected = []
pass_collected = []

# Number of chains of thought to consider for each example
num_chains = 2

for i in tqdm(range(50)):
    cur_question = gsm8k['question'][i]
    cur_answer = gsm8k['answer'][i].split("####")[-1].strip()
    cur_model_input = prompt_chain_of_thought.format(question=cur_question)
    
    # Generate chains of thought
    chains_of_thought = generate_chain_of_thought(cur_question, num_chains)
    
    # Combine chains of thought with the original model input
    cur_model_input_with_chains = f"{cur_model_input}\n\n"
    for j, chain in enumerate(chains_of_thought):
        cur_model_input_with_chains += f"Chain of Thought {j + 1}:\n{chain}\n"
    
    # Assuming generate method of LLM class supports a list of inputs
    result = llama.generate([cur_model_input_with_chains], stop=["\n\n", "[Example 4]"])
    
    if "Error:" in result:
        print(result)
        break
    
    # Extracting the generated answer from the result
    generated_text = result['generations'][0]['text']
    
    # Check if "Answer:" is present in the generated text
    if "Answer:" in generated_text:
        cur_prediction = generated_text.split("Answer:")[1].strip()
        
        # Compare the prediction with the answer
        pass_collected.append(cur_prediction.strip().replace("$", "") == cur_answer)
        
        results_collected.append({
            "question": cur_question,
            "answer": cur_answer,
            "prediction": f"Answer:{cur_prediction}"
        })
    else:
        print(f"Error: 'Answer:' not found in the generated text.")
    
    print(f"Acc: {sum(pass_collected) / len(pass_collected)}")

with open("opt-2.7b.json", "w") as f:
    json.dump(results_collected, f, indent=4)


  2%|▏         | 1/50 [00:00<00:17,  2.75it/s]

Acc: 0.0


  4%|▍         | 2/50 [00:00<00:15,  3.07it/s]

Acc: 0.0


  6%|▌         | 3/50 [00:00<00:15,  3.09it/s]

...

Acc: 0.0


 98%|█████████▊| 49/50 [00:31<00:00,  1.26it/s]

Error: 'Answer:' not found in the generated text.
Acc: 0.0


100%|██████████| 50/50 [00:31<00:00,  1.57it/s]

Acc: 0.0
